## Step 1

Define a function that takes a bound , upper and lower in our occasion and modifies it such as to be in the appropriate format 
e.g. If the value is equal to 330.000, it is contained in the ( 250.000, 500.000) range. The lower bound will be 250000 and the upper bound 500000. Next, the function takes for example the lower bound and transforms it to 250K. The same procedure for the upper bound 500K.

<b>Τhe procedure of transforming a bound is complex, depending on the situation. Thus, we use control flows to cover all the possible cases </b>

In [111]:
import pandas as pd

def modify_bound(bound):           
    if bound < 0: 
        bound = int(str(bound).lstrip("-"))    #if a bound is negative we must reformat such as it does not have the sumbol '-'
        if (bound // 1000000) != 0:        #If the integer division is different from zero that means the bound belongs to millions                 
            bound = bound / 1000000          # we divide it with the million to take the actual million value of the bound
            bound = f"({bound}M)"       # because it is negative and also belongs to millions we add parenthesis and the letter "M" in the end
            if ".0" in bound:        # here we have a detail. We do not use int() function to millions because a bound  of 1.25M would be 1M and we would lose the thousands part which is essential for the accuracy of the interval.Instead we can make the 1.0M object to 1M.
                bound = bound.replace(".0", "")
        elif (bound // 1000) != 0: 
            bound = bound / 1000
            bound = f"({int(bound)}K)" # the differ here is that we are interested only in the thousands part
        else:
            bound = f"({bound})"
    else:                              # the same procedure is done for a positive bound
        if (bound // 1000000) != 0: 
            bound = bound / 1000000
            bound = f"{bound}M"
            if ".0" in bound:
                bound = bound.replace(".0", "")
        elif (bound // 1000) != 0:
            bound = bound / 1000
            bound = f"{int(bound)}K"
        else:
            bound = f"{bound}"
    return bound

## Step 2


Define a function that assigns a range label for the corresponding value

In [112]:
def assign_range_label(value):
    width = 250000 # we define the width of the ranges
    if value > 3000000:
        return ">3M"
    elif value < -3000000:
        return "<-3M"
    else:
        lower_bound = (value // width) * width # the lower bound is equal to the immediate smaller value which is multiple of the width
        upper_bound = lower_bound + width
        modified_lower_bound = modify_bound(lower_bound) # we use the modify_bound function to transform the bounds
        modified_upper_bound = modify_bound(upper_bound)
        label = f"{modified_lower_bound} - {modified_upper_bound}" # here we assign the complete format of the range
        return label

## Step 3

In this step we read the excel variable values into a tabular form, DataFrame.

In [113]:
df = pd.read_excel("Variable Ranges Assessment Data.xlsx")
modified_df = pd.DataFrame()
modified_df['company'] = df['company'] # In the new modified dataframe we create the first column because its' values remain the same as they are categorical and they refer to a specific company

## Step 4 
We loop through the columns of the df DataFrame to replace ","  with " " from  all the values and after that we use the function <b> assign_range_label()</b> to label the ranges of the values


In [117]:
for column in df.columns[1:]:
    modified_df[column] = df[column].astype(str).str.replace("," , "").astype(float).astype(int)
    modified_df[column] = modified_df[column].apply(lambda value: assign_range_label(value)) 

Print the Original DataFrame

In [118]:
df

,company,variable1,variable2,variable3,variable4,variable5,variable6,variable7,variable8
0,Company1,1.084708e+10,7.428960e+08,5.399475e+09,1.663540e+08,4.395829e+09,95473000.0,4.736317e+09,-2.323677e+09
1,Company2,9.950000e+04,-1.454960e+08,2.760500e+07,-1.848770e+08,2.648200e+07,-213763000.0,2.692400e+07,-4.614810e+08
2,Company3,3.229167e+05,8.677570e+03,2.389205e+05,3.559598e+04,1.035483e+05,-11418.5,3.239719e+05,-2.929861e+08
3,Company4,3.924580e+08,-4.229700e+07,3.290640e+08,-1.791760e+08,3.664860e+08,-103610000.0,6.656380e+08,-1.328900e+08
4,Company5,2.333904e+08,4.105327e+07,2.055261e+08,4.941022e+07,1.395148e+05,-34729250.0,1.790086e+05,-6.071164e+07
...,...,...,...,...,...,...,...,...,...
2995,Company2996,6.250211e+06,6.567330e+05,5.684321e+06,-1.008340e+05,5.949550e+06,-196037.0,6.818392e+06,2.710710e+05
2996,Company2997,3.517622e+06,2.746558e+05,3.682356e+06,2.718153e+05,3.823168e+06,272534.6,3.899224e+06,2.721052e+05
2997,Company2998,3.977025e+06,4.701618e+05,5.124930e+06,4.365256e+05,4.339710e+06,303418.3,3.752597e+06,2.721375e+05
2998,Company2999,2.805924e+06,3.986982e+05,1.261682e+06,1.824551e+05,3.470517e+05,-435445.8,1.759994e+06,2.722234e+05


Print the new Modified DataFrame

In [119]:
modified_df

,company,variable1,variable2,variable3,variable4,variable5,variable6,variable7,variable8
0,Company1,<-3M,>3M,<-3M,>3M,<-3M,>3M,<-3M,<-3M
1,Company2,0 - 250K,<-3M,>3M,<-3M,>3M,<-3M,>3M,<-3M
2,Company3,250K - 500K,0 - 250K,0 - 250K,0 - 250K,0 - 250K,(250K) - 0,250K - 500K,<-3M
3,Company4,>3M,<-3M,>3M,<-3M,>3M,<-3M,>3M,<-3M
4,Company5,>3M,>3M,>3M,>3M,0 - 250K,<-3M,0 - 250K,<-3M
...,...,...,...,...,...,...,...,...,...
2995,Company2996,>3M,500K - 750K,>3M,(250K) - 0,>3M,(250K) - 0,>3M,250K - 500K
2996,Company2997,>3M,250K - 500K,>3M,250K - 500K,>3M,250K - 500K,>3M,250K - 500K
2997,Company2998,>3M,250K - 500K,>3M,250K - 500K,>3M,250K - 500K,>3M,250K - 500K
2998,Company2999,2.75M - 3M,250K - 500K,1.25M - 1.5M,0 - 250K,250K - 500K,(500K) - (250K),1.75M - 2M,250K - 500K


Lastly, we can store these results if we want to a new Excel. We can define the path for the output file.

*e.g.*

<b>output_excel_path = "C:/../../Desktop/Variable Ranges/modified_data.xlsx"</b>

<b>modified_df.to_excel(output_excel_path, index=False)</b>